In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

2023-11-19 02:38:37.855671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-19 02:38:37.855700: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


#### Data Preparation

In [2]:
#! wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/bee-wasp-data/data.zip

In [3]:
#!unzip data.zip

#### Model

- Creation

In [4]:
model = models.Sequential()

inp_shape = (150, 150, 3)

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=inp_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

2023-11-19 02:38:39.021456: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-19 02:38:39.021485: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-19 02:38:39.021503: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-1): /proc/driver/nvidia/version does not exist
2023-11-19 02:38:39.021705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


- Compilation

In [5]:
optimizer = optimizers.SGD(learning_rate=0.002, momentum=0.8)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])

- Summary

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

#### Image Datageneeator

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

- Train generator

In [8]:
train_generator = train_datagen.flow_from_directory('data/train/',
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 3677 images belonging to 2 classes.


- Validation Generator

In [9]:
test_generator = test_datagen.flow_from_directory('data/test/',
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        shuffle=True,
                                                        class_mode='binary', )

Found 918 images belonging to 2 classes.


- Training the model

In [10]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator)

Epoch 1/10
  1/184 [..............................] - ETA: 1:08 - loss: 0.6833 - acc: 0.6000

184/184 [==============================] - 15s 78ms/step - loss: 0.6942 - acc: 0.5344 - val_loss: 0.6747 - val_acc: 0.5414
Epoch 2/10
184/184 [==============================] - 14s 78ms/step - loss: 0.6735 - acc: 0.5722 - val_loss: 0.6523 - val_acc: 0.5545
Epoch 3/10
184/184 [==============================] - 14s 78ms/step - loss: 0.6593 - acc: 0.6035 - val_loss: 0.6251 - val_acc: 0.6416
Epoch 4/10
184/184 [==============================] - 14s 78ms/step - loss: 0.6433 - acc: 0.6190 - val_loss: 0.6874 - val_acc: 0.5468
Epoch 5/10
184/184 [==============================] - 14s 77ms/step - loss: 0.6300 - acc: 0.6413 - val_loss: 0.5938 - val_acc: 0.6972
Epoch 6/10
184/184 [==============================] - 14s 77ms/step - loss: 0.5907 - acc: 0.6889 - val_loss: 0.5701 - val_acc: 0.7168
Epoch 7/10
184/184 [==============================] - 14s 77ms/step - loss: 0.5514 - acc: 0.7261 - val_loss: 0.5689 - val_acc: 0.7004
Epoch 8/10
184/184 [==============================] - 14s 77ms/step - los

In [11]:
acc_median = np.median(history.history['acc'])
acc_median

0.6650802195072174

In [12]:
loss_std = np.std(history.history['loss'])
loss_std

0.07311584656464237

#### DATA AUGMENTATION

In [15]:
train_datagen = ImageDataGenerator(
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

In [16]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
train_generator = train_datagen.flow_from_directory('data/train/',
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    shuffle=True,
                                                    class_mode='binary')

Found 3677 images belonging to 2 classes.


In [18]:
validation_generator = test_datagen.flow_from_directory(
    'data/test/',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')

Found 918 images belonging to 2 classes.


In [19]:
#@ MODEL TRAINING WITH AUGMENTATION:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10
184/184 [==============================] - 19s 103ms/step - loss: 35.5214 - acc: 0.5132 - val_loss: 0.6932 - val_acc: 0.5370
Epoch 2/10
184/184 [==============================] - 19s 104ms/step - loss: 0.6921 - acc: 0.5374 - val_loss: 0.6914 - val_acc: 0.5370
Epoch 3/10
184/184 [==============================] - 19s 103ms/step - loss: 0.6914 - acc: 0.5374 - val_loss: 0.6908 - val_acc: 0.5370
Epoch 4/10
184/184 [==============================] - 19s 104ms/step - loss: 0.6912 - acc: 0.5374 - val_loss: 0.6906 - val_acc: 0.5370
Epoch 5/10
184/184 [==============================] - 19s 103ms/step - loss: 0.6904 - acc: 0.5374 - val_loss: 0.6906 - val_acc: 0.5370
Epoch 6/10
184/184 [==============================] - 19s 104ms/step - loss: 0.6900 - acc: 0.5374 - val_loss: 0.6905 - val_acc: 0.5370
Epoch 7/10
184/184 [==============================] - 19s 104ms/step - loss: 0.6906 - acc: 0.5374 - val_loss: 0.6904 - val_acc: 0.5370
Epoch 8/10
184/184 [==============================] - 

In [20]:
val_acc = history.history['val_acc']

val_loss = history.history['val_loss']

In [22]:
np.mean(val_loss)

0.6908375501632691

In [23]:
val_acc

[0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427]

In [24]:
last_5_acc = val_acc[5:10]
last_5_acc

[0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427,
 0.5370370149612427]

In [25]:
np.mean(last_5_acc)

0.5370370149612427